<h1><center><b> Comparison of A* and Dijkstra Algorithms for Pathfinding </b></center></h1>


### **Introduction**

The provided code implements and compares two popular shortest-path algorithms, **A**\* (A-Star) and **Dijkstra**. These algorithms are evaluated based on runtime, number of nodes visited, and path length. The analysis is conducted using OpenStreetMap (OSM) data for a road network between the **Golden Gate Bridge** 🌉 in San Francisco, CA, and **Etcheverry Hall** 🐻 in UC Berkeley, CA. The algorithms are visualized using Folium maps to show both explored nodes and optimal paths.


### **Description of Algorithms**

#### **A\* Algorithm**

The A\* algorithm is a graph traversal and path search technique that efficiently finds the shortest path by combining the benefits of Dijkstra's algorithm and heuristic estimation:

- **Cost Function**: The algorithm minimizes , where:
  - Actual cost from the start node to the current node .
  - Heuristic estimate of the cost from the current node  to the goal.
- **Heuristic**: The code uses the **Haversine formula** to calculate the great-circle distance between two geographical points. This provides an admissible and consistent heuristic for road networks.
- **Priority Queue**: The nodes are prioritized using their total cost , enabling exploration of promising nodes first.

##### **Haversine Formula**
The Haversine formula is used in pathfinding to calculate the great-circle distance between two points on Earth, accounting for its curvature. It is accurate, computationally efficient, and widely applicable in navigation, logistics, and location-based services.
$$
a = \sin^2\left(\frac{\Delta \phi}{2}\right) + \cos(\phi_1) \cdot \cos(\phi_2) \cdot \sin^2\left(\frac{\Delta \lambda}{2}\right)
$$

$$
c = 2 \cdot \arctan2\left(\sqrt{a}, \sqrt{1-a}\right)
$$

$$
d = R \cdot c
$$

Variables:
- ⲫ: Latitudes (in radians)
- λ: Longitudes (in radians)
- R = 6,371 km: Earth's radius

In the code:

- Nodes are tracked using a `heapq` priority queue.
- Explored nodes and their parents are stored in a dictionary to reconstruct the path.
- The algorithm terminates once the goal node is reached, returning the optimal path.

#### **Dijkstra's Algorithm**

Dijkstra's algorithm is a classic shortest-path algorithm that explores nodes in order of increasing distance from the start node:

- It considers only the **actual cost**  from the start node.
- Unlike A\*, Dijkstra does not use any heuristic function.
- It guarantees the shortest path by exploring all nodes with the smallest cumulative cost first.

In the code:

- NetworkX's `single_source_dijkstra_path_length` computes the path lengths and tracks visited nodes.
- The optimal path is retrieved using `shortest_path` with the edge weight parameter set to `length`.


### **Heuristics and Techniques**

1. **Graph Representation**: The road network is represented as a directed graph using OSMnx, where:
   - Nodes represent intersections or points of interest.
   - Edges represent road segments with weights corresponding to their length.
2. **Haversine Distance**: This heuristic is applied in the A\* algorithm to estimate the distance between nodes, which is critical for speeding up the search by guiding it towards the goal.
3. **Path Tracking**: A custom `PathTracker` class is used to monitor all visited nodes during execution. This enables comparison of explored nodes between A\* and Dijkstra.


### **Experimental Results**

The algorithms were executed on the road network graph to find the shortest distance between Golden Gate Bridge and Etchverry Hall. Key metrics such as runtime, number of nodes visited, and path length were compared. The results are summarized in the table below:


| Algorithm    | Runtime (s) | Visited Nodes | Path Length (nodes) |
| ------------ | ----------- | ------------- | ------------------- |
| **A**\*      |   0.007976  |      507      |         171         |
| **Dijkstra** |   0.425721  |     30,015     |         177         |



<img src="dij.png" width="500" height="350" style="display:inline-block; margin-right:30px;"/>
<img src="a_star.png" width="500" height="350" style="display:inline-block;"/>
&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp  Dijkstra's Algorithm      &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp  &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp  &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp A* Algorithm

- **Yellow Lines**: Represent all nodes explored during the search process.
- **Blue Lines**: Highlight the optimal paths returned by each algorithm.
- **Markers**: Start and end points are marked with green and red icons, respectively.

#### **Observations**:

- **A**\* performs better than Dijkstra in terms of runtime due to its heuristic-driven approach, which reduces the number of nodes visited.
- **A**\* returns the shorter path length, confirming the optimality of A\*.
- Dijkstra explores significantly more nodes because it lacks a heuristic, leading to more exhaustive traversal of the graph.


### **Visualization Insights**

1. **A**\*:
   - The yellow paths are more focused and directional towards the goal due to the Haversine heuristic.
   - This targeted exploration reduces the computational overhead.
2. **Dijkstra**:
   - The yellow paths are more widespread, indicating a less targeted approach.
   - Dijkstra explores all possible paths until the goal is reached, increasing the number of visited nodes.

### **Conclusion**

The comparison between A\* and Dijkstra highlights the efficiency of heuristic-driven search methods for pathfinding:

- **A**\* is faster and visits fewer nodes, making it more suitable for large-scale networks.
- **Dijkstra**, while guaranteed to find the optimal path, incurs higher computational costs due to its exhaustive nature.

By leveraging the Haversine distance as a heuristic, A* effectively reduces unnecessary exploration and accelerates the search process, making it an ideal choice for real-world road networks which is used in Google Maps and has enormous applications in logistics.

<div style="page-break-after: always;"></div>

# Python Code

In [ ]:
import osmnx as ox
import networkx as nx
import folium
import time
from IPython.display import display
from prettytable import PrettyTable

# Define start and end locations
start_address = "Golden Gate Bridge, San Francisco, CA"
end_address = "2505 Hearst Ave, Berkeley, CA 94709"

# Geocode the start and end locations
start_point = ox.geocode(start_address)
end_point = ox.geocode(end_address)

# Download the graph for the area
G = ox.graph_from_point(start_point, dist=20000, network_type='drive')

# Find the nearest nodes to the start and end points
start_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
end_node = ox.distance.nearest_nodes(G, end_point[1], end_point[0])

# Tracking steps during A* and Dijkstra
class PathTracker:
    def __init__(self):
        self.visited = set()

def a_star_path_with_tracking(graph, start, end, weight='length'):
    from heapq import heappop, heappush
    from itertools import count

    def haversine(lat1, lon1, lat2, lon2):
        from math import radians, cos, sin, sqrt, atan2
        R = 6371000  # Earth radius in meters
        phi1, phi2 = radians(lat1), radians(lat2)
        dphi = radians(lat2 - lat1)
        dlambda = radians(lon2 - lon1)
        a = sin(dphi / 2) ** 2 + cos(phi1) * cos(phi2) * sin(dlambda / 2) ** 2
        return R * 2 * atan2(sqrt(a), sqrt(1 - a))

    tracker = PathTracker()
    push = heappush
    pop = heappop

    c = count()
    queue = [(0, next(c), start, 0, None)]
    enqueued = {}
    explored = {}

    while queue:
        _, __, curnode, dist, parent = pop(queue)

        if curnode == end:
            path = [curnode]
            node = parent
            while node is not None:
                path.append(node)
                node = explored[node]
            path.reverse()
            return path, tracker.visited

        if curnode in explored:
            continue

        tracker.visited.add(curnode)
        explored[curnode] = parent

        for neighbor in graph.neighbors(curnode):
            if neighbor in explored:
                continue
            ncost = dist + graph[curnode][neighbor].get(weight, 1)
            if neighbor in enqueued:
                qcost, h = enqueued[neighbor]
                if qcost <= ncost:
                    continue
            else:
                h = haversine(graph.nodes[neighbor]['y'], graph.nodes[neighbor]['x'], graph.nodes[end]['y'], graph.nodes[end]['x'])
            enqueued[neighbor] = ncost, h
            push(queue, (ncost + h, next(c), neighbor, ncost, curnode))

    raise nx.NetworkXNoPath(f"Node {end} not reachable from {start}")

def dijkstra_path_with_tracking(graph, start, end, weight='length'):
    tracker = PathTracker()
    
    path_length_dicts = nx.single_source_dijkstra_path_length(graph, start_node)
    
    for node in path_length_dicts.keys():
        tracker.visited.add(node)

    path = nx.shortest_path(graph, source=start_node, target=end_node, weight=weight)
    
    return path, tracker.visited

# Measure runtime and track visited nodes for A*
start_time = time.time()
path_a_star, visited_a_star = a_star_path_with_tracking(G, start_node, end_node)
a_star_time = time.time() - start_time

# Measure runtime and track visited nodes for Dijkstra
start_time = time.time()
path_dijkstra, visited_dijkstra = dijkstra_path_with_tracking(G, start_node, end_node)
dijkstra_time = time.time() - start_time

# Tabular comparison of metrics
table = PrettyTable()
table.field_names = ["Algorithm", "Runtime (s)", "Visited Nodes", "Path Length (nodes)"]
table.add_row(["A*", f"{a_star_time:.6f}", len(visited_a_star), len(path_a_star)])
table.add_row(["Dijkstra", f"{dijkstra_time:.6f}", len(visited_dijkstra), len(path_dijkstra)])

print(table)

# Create Folium maps for A* and Dijkstra separately with minimalistic tiles (e.g., CartoDB Positron)
route_map_a_star = folium.Map(location=[start_point[0], start_point[1]], zoom_start=13,
                              tiles='CartoDB Positron', attr='Map tiles by CartoDB', control_scale=True)
route_map_dijkstra = folium.Map(location=[start_point[0], start_point[1]], zoom_start=13,
                                tiles='CartoDB Positron', attr='Map tiles by CartoDB', control_scale=True)

# Add visited paths for A* in yellow on its map
for node in visited_a_star:
    neighbors_a_star = list(G.neighbors(node))
    for neighbor in neighbors_a_star:
        if neighbor in visited_a_star:
            folium.PolyLine([(G.nodes[node]['y'], G.nodes[node]['x']), 
                             (G.nodes[neighbor]['y'], G.nodes[neighbor]['x'])],
                            color="yellow", weight=2).add_to(route_map_a_star)

# Add visited paths for Dijkstra in yellow on its map
for node in visited_dijkstra:
    neighbors_dijkstra = list(G.neighbors(node))
    for neighbor in neighbors_dijkstra:
        if neighbor in visited_dijkstra:
            folium.PolyLine([(G.nodes[node]['y'], G.nodes[node]['x']), 
                             (G.nodes[neighbor]['y'], G.nodes[neighbor]['x'])],
                            color="yellow", weight=2).add_to(route_map_dijkstra)

# Highlight the shortest path for A* in blue on its map
route_coords_a_star = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path_a_star]
folium.PolyLine(route_coords_a_star, color="blue", weight=5).add_to(route_map_a_star)

# Highlight the shortest path for Dijkstra in blue on its map (should be same as A*)
route_coords_dijkstra = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path_dijkstra]
folium.PolyLine(route_coords_dijkstra, color="blue", weight=5).add_to(route_map_dijkstra)

# Add start and end points on both maps
for route_map in [route_map_a_star, route_map_dijkstra]:
    folium.Marker(location=[start_point[0], start_point[1]], popup="Start", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(location=[end_point[0], end_point[1]], popup="End", icon=folium.Icon(color="red")).add_to(route_map)

# Display both maps separately
display(route_map_a_star)
display(route_map_dijkstra)